In [1]:
anova_all.py

NameError: name 'anova_all' is not defined

In [2]:
./anova_all.py

SyntaxError: invalid syntax (<ipython-input-2-70bb34504706>, line 1)

In [3]:

def get_gesture_counts(participant):
	gestures = {"drag":0,
				"draw":0,
				"ui":0,
				"tap":0,
				"doubletap":0,
				"tripletap":0,
				"hold":0,
				"pinch":0,
				"rev_pinch":0,
				"lasso":0,
				"box":0,
				"voice":0,
				"other":0}

	#Get all the events
	events = []
	for task in participant["tasks"].keys():
		events.extend(participant["tasks"][task])

	for event in events:
	 	if event["event_type"] == "tap":
	 		#Taps need special handling, as they might be double, triple, or hold
	 		if event["hold"]:
	 			gestures["hold"] += 1
	 		elif event["count"] == 2:
	 			gestures["doubletap"] += 1
	 		elif event["count"] == 3:
	 			gestures["tripletap"] += 1
	 		else:
	 			gestures["tap"] += 1
		elif event["event_type"] == "drag":
			#Drags might be drag or might be draw
			if event["draw"] is None:
				gestures["drag"] += 1
			else:
				gestures["draw"] += 1
		elif event["event_type"] == "pinch":
			#pinch can be pinch or reverse
			if event["reverse"]:
				gestures["rev_pinch"] += 1
			else:
				gestures["pinch"] += 1
		elif event["event_type"] == "voice_command":
			gestures["voice"] += 1
		elif event["event_type"] == "ui":
			gestures["ui"] += 1
		elif event["event_type"] == "memo":
			#Don't do anything with memos
			pass
		elif event["event_type"] == "lasso":
			gestures["lasso"] += 1
		elif event["event_type"] == "box_select":
			gestures["box"] += 1
		elif event["event_type"] == "other":
			gestures["other"] += 1		
		else:
			#This is an error, some event type wasn't handled
			print event["event_type"]

	return gestures

In [4]:
def average_counts(counts):
	participant_count = len(counts.keys())
	totals = {}
	#Collect the total counts
	for p in counts.keys():
		p_count = counts[p]
		for gesture in p_count.keys():
			if gesture in totals.keys():
				totals[gesture] += p_count[gesture]
			else:
				totals[gesture] = p_count[gesture]
	#Average across participants
	for gesture in totals.keys():
		totals[gesture] = totals[gesture]/float(participant_count)

	return totals

In [5]:
import all_data_handler
import pandas

In [6]:
adh = all_data_handler.UserData()

What I actually want to do here is have a dataframe where each user is also tagged with what condition they were in, so I can run ANOVA on that, with the condition as my categorical variable. 

In [92]:
data = []
for condition in adh.conditionMap.keys():
    counts = adh.applyCondition(get_gesture_counts, condition)
    #Convert to a list of dicts with user as a parameter of the dictionary
    #First put the user ID and the condition in the data
    for entry in counts:
        counts[entry]["user"] = entry
        counts[entry]["condition"] = condition
        #Tag as a multi-robot or more-or-less single robot condition
        if condition == "one" or condition == "unknown":
            counts[entry]["multi"] = False
        else:
            counts[entry]["multi"] = True
    data.extend(counts.values())

In [93]:
df = pandas.DataFrame(data)

In [94]:
df.set_index("user")

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0,unknown,0,19,3,1,0,False,1,1,0,0,0,0,2
26,0,unknown,5,25,4,2,0,False,1,0,4,4,1,0,0
21,0,unknown,0,13,3,2,1,False,1,1,0,2,0,0,0
16,0,unknown,0,34,1,0,0,False,0,0,0,0,0,0,0
1,0,unknown,7,28,5,5,0,False,0,1,2,2,0,0,0
6,0,unknown,0,20,18,0,0,False,2,1,3,32,0,0,0
25,0,thousand,0,17,5,0,0,True,0,1,0,0,0,0,8
15,0,thousand,0,36,21,0,10,True,0,0,2,0,0,0,2
10,0,thousand,0,18,1,0,0,True,21,0,0,1,0,0,1


So that gets my data into a nice frame, now how do I tell jupyter to do ANOVA to it?

In [52]:
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [95]:
model = ols('drag ~ multi', data=df).fit()

In [96]:
table = sm.stats.anova_lm(model, typ=1)

In [97]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,2376.200000,2376.200000,4.12045,0.051967
Residual,28,16147.166667,576.684524,NaN,NaN


I have two problems here. The first is that I'm not sure that I'm expressing the dependence between the condition and the variable correctly, and the second is that I don't know how to interpret the output. I think that a low PR(>F) is a good thing, but I'm not sure how low is good enough to say that a given gesture is related to the condition. 

In [98]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   drag   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     4.120
Date:                Mon, 23 Apr 2018   Prob (F-statistic):             0.0520
Time:                        20:32:14   Log-Likelihood:                -136.89
No. Observations:                  30   AIC:                             277.8
Df Residuals:                      28   BIC:                             280.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept        17.6667      6.932      2.548

I think that what this means is that I can accept that drag is conditional on condition with 1-0.0123 = 0.9877 likelyhood

In [99]:
model = ols('lasso ~ multi', data=df).fit()

In [100]:
table = sm.stats.anova_lm(model, typ=1)

In [101]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,341.688889,341.688889,4.117127,0.052055
Residual,28,2323.777778,82.992063,NaN,NaN


In [102]:
model = ols('lasso ~ condition', data=df).fit()

In [103]:
table = sm.stats.anova_lm(model, typ=1)

In [104]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,592.800000,148.200000,1.787552,0.162909
Residual,25,2072.666667,82.906667,NaN,NaN


This makes it seem like whether the condition is a multi-robot condition is a stronger predictor (0.052 vs 0.163) of the use of lasso than what the actual condition is.